In [ ]:
from model.booster import HistBooster

In [ ]:
import pandas as pd
import numpy as np

from hydra.utils import instantiate
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra

from data_pipeline.pipelinesteps import data_splitter
from model.modelorchastrator import ModelOrchestrator
from data_pipeline.dataset import Dataset
from evaluate.metric_eval import ModelEvaluator



# Create config
GlobalHydra.instance().clear()
initialize(config_path="../../conf", version_base=None)
cfg = compose(config_name="config")

# Create Dataset
df = pd.read_feather("../../data/parquet_files/train/processed_train.feather")
df.pop('date_decision')
data_pipeline = instantiate(cfg.data_pipeline)
df = data_pipeline.apply(df)  

dataset = Dataset(data=df, data_splitter=data_splitter, target_column='target')

In [ ]:
cfg.data_pipeline

In [ ]:
dataset.X_train['Age']

In [ ]:
model_orchestrator = ModelOrchestrator(cfg)

model = model_orchestrator.create_pipe_line()
model.fit(dataset.X_train, dataset.y_train)

In [ ]:
clf = HistBooster()
clf.fit(model.transform_without_predictor(dataset.X), dataset.y)

In [ ]:
clf.feature_importances_

In [ ]:
sum(clf.feature_importances_)